# Peer-graded Assignment - Week 3

## 1. Install the Beautiful Soup Package

In [1]:
#Install Beautiful Soup 
!conda install -c anaconda beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be UPDATED:

    certifi:         2020.4.5.1-py36h9f0ad1d_0 conda-forge --> 2020.4.5.1-py36_0 anaconda
    openssl:         1.1.1g-h516909a_0         conda-forge --> 1.1.1-h7b6447c_0  anaconda

The following packages will be DOWNGRADED:

    ca-certificates: 2020.4.5.1-hecc5488_0     conda-forge --> 2020.1.1-0        anaconda

Preparing transaction: done
Verifying transaction: done
Executing transaction: done


## 2. Import Beautiful Soup package to web scrape the Wikipedia Web page.

- Read the web page then pass it to the Beautiful Soup html parser.
- Find the table class in the html.
- Iterate through the table rows stripping the text to remove newline characters.
- Only add records where the Borough does NOT equal 'Not assigned' are added.
- If the Neighborhood field is blank then assign it the same value as the Borough field. 
- Change the forward slash characters to commas in the Neigborhood field.
- Create a dataframe.


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Request webpage
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Parse webpage into beautifulsoup object
soup = BeautifulSoup(source,'html.parser')

#Find the table of interest and read it into a dataframe
table = soup.find("table",{"class":"wikitable sortable"})
#df_all = pd.read_html(str(table))[0]

table_rows = table.find_all('tr')
pc_list=[]
for tr in table_rows:
    td = tr.find_all('td')
    pc_row=[]
    pc_row = [tr.text.strip() for tr in td]
    
    
    if len(pc_row) != 0:
        # Replace forward slash with a comma in neighborhood field
        pc_row[2] = pc_row[2].replace(' / ', ', ')
        
        # if Neighborhood field is blank assign the Borough value to it
        if pc_row[2] == '':
            pc_row[2] = pc_row[1]
        
        # do not add row if the Borough value equals Not assigned
        if pc_row[1] != 'Not assigned':
            pc_list.append(pc_row)    



df_pc = pd.DataFrame(pc_list,columns=["PostalCode","Borough","Neighborhood"])
df_pc.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## 3. Print the number of rows

In [3]:
print('No of Rows :', df_pc.shape[0])

No of Rows : 103


## 4. Import the Postal Code lookup csv into a dataframe then rename the Postal Code column to PostalCode

In [9]:
#Read in the csv file from the webpage
df_pc_lookup = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")

#Rename the Postal Code column
df_pc_lookup.rename(columns= {'Postal Code':'PostalCode'}, inplace=True)
df_pc_lookup.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 5. Join the Postal Code dataframe to the Postal Code lookup table to geocode the data.

In [26]:
#Join the postal code and postal code lookup dataframes
df_pc_geocoded = df_pc_lookup.set_index(['PostalCode']).combine_first(df_pc.set_index(['PostalCode'])).reset_index()

#Re-order the columns
df_pc_geocoded = df_pc_geocoded[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
df_pc_geocoded.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
